In [11]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from metrics import group_metrics
from sklearn.preprocessing import OneHotEncoder
from train_clp_adult import train_fair_nn
import tensorflow.compat.v1 as tf
import json
import sys
from data_preprocess import get_data
from sklearn import linear_model
import compas_data as compas

def standardize(x):
    return (x - np.mean(x))/np.std(x)

Instructions for updating:
non-resource variables are not supported in the long term


In [12]:
seed_data, seed_model = 1, 2
# Extracting compas data
x_train, x_test, y_train, y_test, y_sex_train, y_sex_test,\
        y_race_train, y_race_test, _ = compas.get_compas_train_test(random_state = seed_data)
group_train, group_test = x_train[:, :2], x_test[:, :2]
x_train, x_test = x_train[:, 2:], x_test[:, 2:]

group_names = ['sex', 'race']


one_hot = OneHotEncoder(sparse=False)
one_hot.fit(y_train.reshape(-1,1))
names_income = one_hot.categories_
y_train = one_hot.transform(y_train.reshape(-1,1))
y_test = one_hot.transform(y_test.reshape(-1,1))

# Standardizing the last four columns
for i in range(1, 5):
    x_train[:, i] = standardize(x_train[:, i])
    x_test[:, i] = standardize(x_test[:, i])

    # Calculate the sensitive directions
sensetive_directions = []
protected_regression = linear_model.LogisticRegression(fit_intercept = True)
protected_regression.fit(x_train, y_sex_train)
sensetive_directions.append(protected_regression.coef_.reshape((-1,)))
protected_regression.fit(x_train, y_race_train)
sensetive_directions.append(protected_regression.coef_.reshape((-1,)))
sensetive_directions = np.array(sensetive_directions)


(5278, 8)
['sex', 'race', 'priors_count', 'age_cat=25 to 45', 'age_cat=Greater than 45', 'age_cat=Less than 25', 'c_charge_degree=F', 'c_charge_degree=M']
sex
race


In [5]:
tf.reset_default_graph()
fair_info = [group_train, group_test, group_names, sensetive_directions]
weights, train_logits, test_logits, _, variables = train_fair_nn(x_train,\
     y_train,tf_prefix='sensr', adv_epoch_full=50, l2_attack=0.0001,\
         adv_epoch=10, ro=0.001, adv_step=10., plot=False,\
        fair_info=fair_info,balance_batch=True,\
            X_test = x_test, X_test_counter=None,\
                y_test = y_test, lamb_init=2.,\
                n_units=[100], l2_reg=0, epoch=4000,\
                batch_size=1000, lr=1e-4, lambda_clp=0.,\
            fair_start=0., counter_init=False, seed=seed_model)

objective decreased from 0.693042 to 0.691664; resetting the attack
Real and fair distances (max/min/mean):
3570.4347743409508 46.67233662734618 1468.0700969519899
0.000110053035 3.684736e-05 0.000101877486
Counter success count is -1
Epoch 3000 train accuracy 0.497631; loss 0.691744; lambda is 0.000010
Epoch 3000 test accuracy 0.472538
FAILED attacks: subspace 179; full 0; Nans after attack 0
Loss clean 0.691779; subspace 0.694816; adv 0.695788
Real and fair distances (max/min/mean):
4509.161337171704 83.73578885234537 1637.7020247692112
0.000110053035 3.684736e-05 0.000101877486
Counter success count is -1
Epoch 3999 train accuracy 0.545476; loss 0.688767; lambda is 0.000010
Epoch 3999 test accuracy 0.50947
FAILED attacks: subspace 290; full 0; Nans after attack 0
Loss clean 0.686025; subspace 0.691157; adv 0.692147

Final train accuracy 0.545476
Final test accuracy 0.50947
Final lambda 0.000010


In [6]:
print('Gender:')
_ = group_metrics(y_test[:,1], test_logits.argmax(axis=1), group_test[:,0],\
     label_protected=0,label_good=1)
print('\nRace:')
_ = group_metrics(y_test[:,1], test_logits.argmax(axis=1), group_test[:,1],\
     label_protected=0, label_good=1)

Gender:
Accuracy is 0.509470
Balanced accuracy is 0.518150
Gap RMS is 0.147684461869415
Mean absolute gap is 0.10483439733726196
Max gap is 0.2088557865444427
Average odds difference is 0.104021
Equal opportunity difference is -0.000813
Statistical parity difference is 0.129000

Race:
Accuracy is 0.509470
Balanced accuracy is 0.518150
Gap RMS is 0.11472973115977106
Mean absolute gap is 0.11467600472884776
Max gap is 0.11818672423097676
Average odds difference is 0.114676
Equal opportunity difference is 0.118187
Statistical parity difference is 0.117180


In [7]:
y_test

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [8]:
test_logits

array([[ 0.10920897,  0.32617792],
       [ 0.05807625,  0.07834068],
       [-0.08675189,  0.14817993],
       ...,
       [ 0.09307206,  0.318889  ],
       [-0.11403552,  0.12835373],
       [ 0.09701891, -0.0232924 ]], dtype=float32)

In [9]:
test_logits.argmax(axis=1)

array([1, 1, 1, ..., 1, 1, 0])

In [10]:
y_true, y_pred, y_protected = y_test[:, 1], test_logits.argmax(axis=1), group_test[:, 0]
label_protected,label_good= 0, 1

In [14]:
idx_prot = np.where(y_protected == label_protected)[0]
idx_priv = np.where(y_protected != label_protected)[0]
idx_good_class = np.where(y_true == label_good)[0]
idx_pred_good_class = np.where(y_pred == label_good)[0]
idx_bad_class = np.where(y_true != label_good)[0]
idx_pred_bad_class = np.where(y_pred != label_good)[0]
correct = y_true==y_pred
	
TPR_prot = correct[np.intersect1d(idx_good_class, idx_prot)].mean()
FP_prot = (1-correct[np.intersect1d(idx_pred_good_class, idx_prot)]).sum()
FPR_prot = FP_prot/len(np.intersect1d(idx_bad_class, idx_prot))
TPR_priv = correct[np.intersect1d(idx_good_class, idx_priv)].mean()
FP_priv = (1-correct[np.intersect1d(idx_pred_good_class, idx_priv)]).sum()
FPR_priv = FP_priv/len(np.intersect1d(idx_bad_class, idx_priv))
    
accuracy = correct.mean()
print('Accuracy is %f' % accuracy)
    
bal_acc = (correct[idx_good_class].mean() + correct[idx_bad_class].mean())/2
print('Balanced accuracy is %f' % bal_acc)
    
    # TPR_bad_prot = correct[np.intersect1d(idx_bad_class, idx_prot)].mean()
    # TPR_bad_priv = correct[np.intersect1d(idx_bad_class, idx_priv)].mean()
gaps = np.array([np.abs(TPR_prot - TPR_priv), np.abs(FPR_prot - FPR_priv)])
gap_rms = np.sqrt((gaps**2).mean())
mean_gap = gaps.mean()
max_gap = gaps.max()
print('Gap RMS is', gap_rms)
print('Mean absolute gap is', mean_gap)
print('Max gap is', max_gap)
    
average_odds_difference = ((TPR_prot - TPR_priv) + (FPR_prot - FPR_priv))/2
print('Average odds difference is %f' % average_odds_difference)
    
equal_opportunity_difference = TPR_prot - TPR_priv
print('Equal opportunity difference is %f' % equal_opportunity_difference)
    
statistical_parity_difference =\
     (y_pred[idx_prot]==label_good).mean()- (y_pred[idx_priv]==label_good).mean()
print('Statistical parity difference is %f' % statistical_parity_difference)

Accuracy is 0.509470
Balanced accuracy is 0.517146
Gap RMS is nan
Mean absolute gap is nan
Max gap is nan
Average odds difference is nan
Equal opportunity difference is nan
Statistical parity difference is nan


In [15]:
gaps

array([nan, nan])

In [16]:
TPR_prot

nan

In [17]:
idx_prot

array([], dtype=int64)

In [18]:
y_protected

array([ 1.94544287, -0.70866907, -0.30034415, ...,  1.74128041,
       -0.50450661, -0.70866907])

In [15]:
import numpy as np
import itertools
starts = np.arange(0, 901, 100)
ends = np.arange(100, 1001, 100)
expts = ['sensr', 'reduction', 'baseline', 'baseline_bal'] 
data_index = range(ends.shape[0])
iteration = range(10)
lrs = [2e-3, 5e-3, 1e-2]

a = itertools.product(expts, data_index, iteration, lrs)
b = [i for i in a]

In [16]:
def printf(n):
    n = n* 10
    print(f'{b[n + 3]} {b[n+2]} {b[n+4]}')

In [17]:
for i in [1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 30]:
    printf(i)

('sensr', 0, 4, 0.005) ('sensr', 0, 4, 0.002) ('sensr', 0, 4, 0.01)
('sensr', 1, 4, 0.005) ('sensr', 1, 4, 0.002) ('sensr', 1, 4, 0.01)
('sensr', 2, 4, 0.005) ('sensr', 2, 4, 0.002) ('sensr', 2, 4, 0.01)
('sensr', 3, 4, 0.005) ('sensr', 3, 4, 0.002) ('sensr', 3, 4, 0.01)
('sensr', 4, 4, 0.005) ('sensr', 4, 4, 0.002) ('sensr', 4, 4, 0.01)
('sensr', 5, 4, 0.005) ('sensr', 5, 4, 0.002) ('sensr', 5, 4, 0.01)
('sensr', 6, 4, 0.005) ('sensr', 6, 4, 0.002) ('sensr', 6, 4, 0.01)
('sensr', 7, 4, 0.005) ('sensr', 7, 4, 0.002) ('sensr', 7, 4, 0.01)
('sensr', 8, 4, 0.005) ('sensr', 8, 4, 0.002) ('sensr', 8, 4, 0.01)
('sensr', 9, 4, 0.005) ('sensr', 9, 4, 0.002) ('sensr', 9, 4, 0.01)
('reduction', 0, 1, 0.002) ('reduction', 0, 0, 0.01) ('reduction', 0, 1, 0.005)


In [18]:
starts = np.arange(0, 901, 10)
ends = np.arange(10, 1001, 10)
expts = ['sensr', 'reduction', 'baseline', 'project'] 
data_index = range(ends.shape[0])
iteration = range(10)
lrs = [2e-3, 5e-3, 1e-2]

a = itertools.product(expts, data_index, iteration, lrs)
b = [i for i in a]

In [19]:
b[74]

('sensr', 2, 4, 0.01)

In [20]:
import numpy as np
import tensorflow as tf
from sklearn import linear_model
import utils
import time
import multiprocessing as mp
import random
import matplotlib.pyplot as plt
import scipy
plt.ioff()
import sys
from data_preprocess import get_data
#tf.compat.v1.enable_eager_execution()

def sample_perturbation(data_point, regularizer = 20, learning_rate = 3e-2, num_steps = 200):
    """
    Calculates ratio between perturbed loss and original loss

    parameters: 
        data_point: tuple of x, y
            x: tensor of shape (d, )
            y: one-hot encoded tensor of shape (2, )
        regularizer (float): regularizer constant for fair metric
        learning_rate (float): step size for gradient ascend
        num_steps (int): number of steps in gradient ascend

    return:
        float; ratio of entropy losses for perturbed and original sample
    """
    x, y = data_point
    x = tf.reshape(x, (1, -1))
    y = tf.reshape(y, (1, -1))
    x_start = x
    for i in range(num_steps):
        with tf.GradientTape() as g:
            g.watch(x)
            prob = graph(x)
            perturb = utils.unprotected_direction(x-x_start, sensetive_directions)
            loss = utils.EntropyLoss(y, prob)  - regularizer  * tf.norm(perturb)**2

        gradient = g.gradient(loss, x)
        x = x + learning_rate * gradient/(i ** (2/3))

    return_loss = utils.EntropyLoss(y, graph(x)) / utils.EntropyLoss(y, graph(x_start))
    print('done')
    return return_loss.numpy()

In [21]:
expt, d, i, lr = b[74]
start = starts[d]
end = ends[d]
np.random.seed(1)
seeds = np.load('../seeds.npy')

In [22]:
data_seed = seeds[i, 0]
expt_seed = seeds[i, 1]

In [29]:
import json
x_train, x_test, y_train, y_test, _, y_sex_test, y_race_test = get_data(data_seed)




sensetive_directions = []
protected_regression = linear_model.LogisticRegression(fit_intercept = True)
protected_regression.fit(x_test.numpy(), y_sex_test)
sensetive_directions.append(protected_regression.coef_.reshape((-1,)))
protected_regression.fit(x_test.numpy(), y_race_test)
sensetive_directions.append(protected_regression.coef_.reshape((-1,)))
sensetive_directions = np.array(sensetive_directions)

sensetive_directions = scipy.linalg.orth(sensetive_directions.T).T
for i, s in enumerate(sensetive_directions):
    #while np.linalg.norm(s) != 1:
    s = s/ np.linalg.norm(s)
    sensetive_directions[i] = s

sensetive_directions = tf.cast(sensetive_directions, dtype = tf.float32)



with open(f'models/data_{data_seed}_{expt_seed}.txt', 'r') as f:
    weight = json.load(f)

weights = [np.array(w) for w in weight]

def graph(x):
    layer1 = SimpleDense([weights[0], weights[1]])
    layer2 = SimpleDense([weights[2], weights[3]])
    out = tf.nn.relu(layer1(x))
    out = layer2(out)
    prob = tf.nn.softmax(out)
    return prob



(5278, 8)
['sex', 'race', 'priors_count', 'age_cat=25 to 45', 'age_cat=Greater than 45', 'age_cat=Less than 25', 'c_charge_degree=F', 'c_charge_degree=M']
sex
race


KeyboardInterrupt: 

In [ ]:
graph

In [26]:
data_seed, expt_seed

(20609, 49100)